In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/elon-musks-tweets-dataset-2022/cleandata.csv
/kaggle/input/elon-musks-tweets-dataset-2022/rawdata.csv
/kaggle/input/elon-musks-tweets-dataset-2022/userinfo.csv


*Import Dependencies*

In [2]:
import os
import sys
import warnings
import matplotlib
import pandas
import numpy
import re
import nltk
from textblob import TextBlob

import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
warnings.filterwarnings("ignore")
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
%matplotlib inline

*Load Dataset*

In [3]:
df = pd.read_csv("/kaggle/input/elon-musks-tweets-dataset-2022/rawdata.csv")

*EDA*

In [4]:
df.head(5)

,Tweets,Retweets,Likes,Date
0,@PeterSchiff 🤣 thanks,209,7021,2022-10-27 16:17:39
1,@ZubyMusic Absolutely,755,26737,2022-10-27 13:19:25
2,Dear Twitter Advertisers https://t.co/GMwHmInPAS,55927,356623,2022-10-27 13:08:00
3,@BillyM2k 👻,802,19353,2022-10-27 02:32:48
4,Meeting a lot of cool people at Twitter today!,9366,195546,2022-10-26 21:39:32


In [5]:
df.tail(5)

,Tweets,Retweets,Likes,Date
3055,@LimitingThe @baglino Just that manganese is a...,171,3173,2022-01-27 22:01:06
3056,@incentives101 @ICRicardoLara Exactly,145,4234,2022-01-27 21:23:20
3057,@ICRicardoLara Your policies are directly resp...,421,6144,2022-01-27 21:13:57
3058,@ICRicardoLara You should be voted out of office,484,7029,2022-01-27 21:12:27
3059,CB radios are free from govt/media control,11302,113429,2022-01-27 21:00:09


In [6]:
df.isnull().sum()

Tweets      0
Retweets    0
Likes       0
Date        0
dtype: int64

In [7]:
df.duplicated().sum()

0

In [8]:
df.columns

Index(['Tweets', 'Retweets', 'Likes', 'Date'], dtype='object')

In [9]:
df.shape

(3060, 4)

*Cleaning Data*

In [10]:
def cleantwt (twt):
  emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)

  twt = re.sub('RT', '', twt) # remove 'RT' from tweets
  twt = re.sub('#[A-Za-z0-9]+', '', twt) # remove the '#' from the tweets
  twt = re.sub('\\n', '', twt) # remove the '\n' character
  twt = re.sub('https?:\/\/\S+', '', twt) # remove the hyperlinks
  twt = re.sub('@[\S]*', '', twt) # remove @mentions
  twt = re.sub('^[\s]+|[\s]+$', '', twt) # remove leading and trailing whitespaces
  twt = re.sub(emoj, '', twt) # remove emojis
  return twt

In [11]:
df['Cleaned_tweets'] = df['Tweets'].apply(cleantwt)

In [12]:
df.head()

,Tweets,Retweets,Likes,Date,Cleaned_tweets
0,@PeterSchiff 🤣 thanks,209,7021,2022-10-27 16:17:39,thanks
1,@ZubyMusic Absolutely,755,26737,2022-10-27 13:19:25,Absolutely
2,Dear Twitter Advertisers https://t.co/GMwHmInPAS,55927,356623,2022-10-27 13:08:00,Dear Twitter Advertisers
3,@BillyM2k 👻,802,19353,2022-10-27 02:32:48,
4,Meeting a lot of cool people at Twitter today!,9366,195546,2022-10-26 21:39:32,Meeting a lot of cool people at Twitter today!


In [13]:
df.tail()

,Tweets,Retweets,Likes,Date,Cleaned_tweets
3055,@LimitingThe @baglino Just that manganese is a...,171,3173,2022-01-27 22:01:06,Just that manganese is an alternative to iron ...
3056,@incentives101 @ICRicardoLara Exactly,145,4234,2022-01-27 21:23:20,Exactly
3057,@ICRicardoLara Your policies are directly resp...,421,6144,2022-01-27 21:13:57,Your policies are directly responsible for the...
3058,@ICRicardoLara You should be voted out of office,484,7029,2022-01-27 21:12:27,You should be voted out of office
3059,CB radios are free from govt/media control,11302,113429,2022-01-27 21:00:09,CB radios are free from govt/media control


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3060 entries, 0 to 3059
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Tweets          3060 non-null   object
 1   Retweets        3060 non-null   int64 
 2   Likes           3060 non-null   int64 
 3   Date            3060 non-null   object
 4   Cleaned_tweets  3060 non-null   object
dtypes: int64(2), object(3)
memory usage: 119.7+ KB


In [15]:
# Removing Empty String from all Rows
df.drop(df[df['Cleaned_tweets'] == ''].index, inplace = True)

In [16]:
df.head()

,Tweets,Retweets,Likes,Date,Cleaned_tweets
0,@PeterSchiff 🤣 thanks,209,7021,2022-10-27 16:17:39,thanks
1,@ZubyMusic Absolutely,755,26737,2022-10-27 13:19:25,Absolutely
2,Dear Twitter Advertisers https://t.co/GMwHmInPAS,55927,356623,2022-10-27 13:08:00,Dear Twitter Advertisers
4,Meeting a lot of cool people at Twitter today!,9366,195546,2022-10-26 21:39:32,Meeting a lot of cool people at Twitter today!
5,Entering Twitter HQ – let that sink in! https:...,145520,1043592,2022-10-26 18:45:58,Entering Twitter HQ – let that sink in!


In [17]:
df.tail()

,Tweets,Retweets,Likes,Date,Cleaned_tweets
3055,@LimitingThe @baglino Just that manganese is a...,171,3173,2022-01-27 22:01:06,Just that manganese is an alternative to iron ...
3056,@incentives101 @ICRicardoLara Exactly,145,4234,2022-01-27 21:23:20,Exactly
3057,@ICRicardoLara Your policies are directly resp...,421,6144,2022-01-27 21:13:57,Your policies are directly responsible for the...
3058,@ICRicardoLara You should be voted out of office,484,7029,2022-01-27 21:12:27,You should be voted out of office
3059,CB radios are free from govt/media control,11302,113429,2022-01-27 21:00:09,CB radios are free from govt/media control


*Subjectivity and Polarity*

In [18]:
def getSubjectivity(twt):
    return TextBlob(twt).sentiment.subjectivity

def getPolarity(twt):
    return TextBlob(twt).sentiment.polarity

In [19]:
df['Subjectivity'] = df['Cleaned_tweets'].apply(getSubjectivity)
df['Polarity'] = df['Cleaned_tweets'].apply(getPolarity)

*Creating Sentiments Labels*

In [20]:
def getSentiments(value):
    if value > 0:
        return 'Positive'
    elif value < 0:
        return 'Negative'
    else:
        return 'Netural'

*Sentiment Analysis*

In [21]:
df['Sentiments'] = df['Polarity'].apply(getSentiments)
df['Sentiments'] = df['Subjectivity'].apply(getSentiments)

In [22]:
df.head()

,Tweets,Retweets,Likes,Date,Cleaned_tweets,Subjectivity,Polarity,Sentiments
0,@PeterSchiff 🤣 thanks,209,7021,2022-10-27 16:17:39,thanks,0.20,0.2000,Positive
1,@ZubyMusic Absolutely,755,26737,2022-10-27 13:19:25,Absolutely,0.90,0.2000,Positive
2,Dear Twitter Advertisers https://t.co/GMwHmInPAS,55927,356623,2022-10-27 13:08:00,Dear Twitter Advertisers,0.00,0.0000,Netural
4,Meeting a lot of cool people at Twitter today!,9366,195546,2022-10-26 21:39:32,Meeting a lot of cool people at Twitter today!,0.65,0.4375,Positive
5,Entering Twitter HQ – let that sink in! https:...,145520,1043592,2022-10-26 18:45:58,Entering Twitter HQ – let that sink in!,0.00,0.0000,Netural


In [23]:
df.tail()

,Tweets,Retweets,Likes,Date,Cleaned_tweets,Subjectivity,Polarity,Sentiments
3055,@LimitingThe @baglino Just that manganese is a...,171,3173,2022-01-27 22:01:06,Just that manganese is an alternative to iron ...,0.000,0.00,Netural
3056,@incentives101 @ICRicardoLara Exactly,145,4234,2022-01-27 21:23:20,Exactly,0.250,0.25,Positive
3057,@ICRicardoLara Your policies are directly resp...,421,6144,2022-01-27 21:13:57,Your policies are directly responsible for the...,0.545,0.18,Positive
3058,@ICRicardoLara You should be voted out of office,484,7029,2022-01-27 21:12:27,You should be voted out of office,0.000,0.00,Netural
3059,CB radios are free from govt/media control,11302,113429,2022-01-27 21:00:09,CB radios are free from govt/media control,0.800,0.40,Positive
